[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sdiciotti/Age-Prediction-Demo/blob/master/Age_prediction.ipynb)

In [ ]:
!pip install shap
import ipywidgets as widgets 
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, KFold, GridSearchCV, cross_validate
from sklearn.svm import SVR, SVC
from sklearn.metrics import mean_absolute_error, accuracy_score
import shap
from xgboost import XGBClassifier
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')


In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/sdiciotti/Age-Prediction-Demo/main/NKI2_data.csv')
features = list(df.columns)
df_plot = df.copy()
df

In [ ]:

a = widgets.Dropdown(
    options=features,
    value=features[1],
    description='X:',
    disabled=False,
)
b = widgets.Dropdown(
    options=features,
    value=features[1],
    description='y:',
    disabled=False,
)
display(a,b)


In [ ]:
plt.figure(dpi=100)
plt.scatter(df_plot[a.value],df_plot[b.value])
plt.xlabel(a.value)
plt.ylabel(b.value)

In [ ]:
print("Dataframe shape before NaN removal:", np.shape(df)[0])

In [ ]:
df.dropna(axis=0, how='any', thresh=None, subset=None, inplace=True)
print("Dataframe shape after NaN removal:", np.shape(df)[0])

In [ ]:
### REGRESSION ###
print('***Regression task')

SEED = 42
outer_n_folds = 5
inner_n_folds = 5
C = [0.1, 1, 10]

X = df.iloc[:,2::]
shap_X = df.iloc[:,2::]
y = df['Age']

print('The whole dataset contains ' + str(np.shape(df)[0]) + ' subjects')
print('The age prediction will be performed using ' + str(np.shape(X)[1]) + ' MRI-derived features')
print() 

In [ ]:
# NestedCV implemented in scikit-learn
outer_cv = KFold(n_splits=outer_n_folds, shuffle=True, random_state=SEED)
inner_cv = KFold(n_splits=inner_n_folds, shuffle=True, random_state=SEED)

clf = SVR(kernel='rbf', degree=3, gamma='scale', coef0=0.0, tol=0.001, C=0.1, epsilon=0.1, shrinking=True, cache_size=200, verbose=0, max_iter=- 1)
p_grid = [{'C': C}]     

X = np.asarray(X)
y = np.asarray(y)

clf_gs = GridSearchCV(clf, param_grid=p_grid, cv=inner_cv, refit='neg_mean_absolute_error', scoring='neg_mean_absolute_error', n_jobs=1, verbose = 4)
nested_score = cross_validate(clf_gs, X=X, y=y, cv=outer_cv, return_train_score=True, return_estimator=True, scoring = 'neg_mean_absolute_error', n_jobs=1)

#print(np.abs(nested_score['train_score']))
#print(np.abs(nested_score['test_score']))
print("Average MAE train:", np.abs(np.mean(nested_score['train_score'])))
print("Average MAE test:", np.abs(np.mean(nested_score['test_score'])))

# SHAP VALUES

In [ ]:
clf_gs.fit(shap_X,y)
model = clf_gs.best_estimator_

In [ ]:
explainer = shap.Explainer(model.predict,shap_X)
shap_values = explainer(shap_X)

In [ ]:
shap.plots.waterfall(shap_values[1], max_display=14)

In [ ]:
shap.plots.beeswarm(shap_values, max_display=14)

In [ ]:
shap.plots.bar(shap_values)

In [ ]:
shap.plots.scatter(shap_values[:,"rh_cortex_CT"], color=shap_values)

# CLASSIFICATION OF THE AGE

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/sdiciotti/Age-Prediction-Demo/main/NKI2_data.csv')
df.dropna(axis=0, how='any', thresh=None, subset=None, inplace=True)


In [ ]:
df["Age"].unique()

In [ ]:
bin_age = []
index = []
for i, age in enumerate(df["Age"]):
    if age <=10: 
        bin_age.append(0)
        index.append(i)
        
    elif age >=12: 
        bin_age.append(1)
        index.append(i)

In [ ]:
df = df.iloc[index]
df["bin_age"] = bin_age
del df["Age"]

In [ ]:
df

In [ ]:
### CLASSIFICATION ###
print('***Classification task')

SEED = 42
outer_n_folds = 5
inner_n_folds = 5
C = [0.1, 1, 10]

X = df.iloc[:,1::]
del X["bin_age"]
shap_X = df.iloc[:,1::]
del shap_X["bin_age"]
y = df['bin_age']

print('The whole dataset contains ' + str(np.shape(df)[0]) + ' subjects')
print('The age prediction will be performed using ' + str(np.shape(X)[1]) + ' MRI-derived features')
print() 

In [ ]:
# NestedCV implemented in scikit-learn
outer_cv = KFold(n_splits=outer_n_folds, shuffle=True, random_state=SEED)
inner_cv = KFold(n_splits=inner_n_folds, shuffle=True, random_state=SEED)

#clf = SVC(degree=3, gamma='scale', coef0=0.0, tol=0.001, C=0.1,  shrinking=True, cache_size=200, verbose=0, max_iter=- 1)
#p_grid = [{'C': C}]     
clf = XGBClassifier(verbosity=0)
gamma = [0.1,0.2,0.3]
p_grid =[{"gamma":gamma}]
X = np.asarray(X)
y = np.asarray(y)

clf_gs = GridSearchCV(clf, param_grid=p_grid, cv=inner_cv, refit='roc_auc', scoring='roc_auc', n_jobs=1, verbose = 4)
nested_score = cross_validate(clf_gs, X=X, y=y, cv=outer_cv, return_train_score=True, return_estimator=True, scoring = 'roc_auc', n_jobs=1)

#print(np.abs(nested_score['train_score']))
#print(np.abs(nested_score['test_score']))
print("Average ROC AUC train:", np.abs(np.mean(nested_score['train_score'])))
print("Average ROC AUC test:", np.abs(np.mean(nested_score['test_score'])))

# SHAP VALUES

In [ ]:
clf_gs.fit(shap_X,y)
model = clf_gs.best_estimator_

In [ ]:
explainer = shap.TreeExplainer(model,shap_X,feature_perturbation='interventional', model_output='probability')
shap_values = explainer(shap_X)

In [ ]:
shap.plots.waterfall(shap_values[1], max_display=5)

In [ ]:
shap.plots.beeswarm(shap_values, max_display=14)

In [ ]:
shap.plots.bar(shap_values)

In [ ]:
shap.plots.scatter(shap_values[:,"lh_cerebralGM_FD"], color=shap_values)